In [8]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:

import numpy as np
print(np.__version__)
from mirage import WordCountingChunkingAlgorithm, FolderRawStorage, WhooshChunkStorage
from mirage import HuggingFaceEmbedder, TextNormalizer
from mirage import NatashaSentenceChunking, SemanticChunking
import os
import pickle

1.26.4


In [11]:
embedder = HuggingFaceEmbedder(model_name='BAAI/bge-m3', normalizer=True)

In [12]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data_test')
 # тут заменить на нужную папку
chunks = WhooshChunkStorage(scoring_function="BM25F", normalizer=True)
# algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)


In [13]:

algorithm = SemanticChunking(raw_storage=documents, chunk_storage=chunks, embedder=embedder, threshold=0.5, max_chunk_size=1000)
algorithm.execute(visualize=True)

Splitting text into the sentences


611it [00:00, 50276.02it/s]


creating embeddings of the sentences for semantic grouping


100%|██████████| 611/611 [00:58<00:00, 10.53it/s]


Adding chunks to the storage


100%|██████████| 427/427 [00:02<00:00, 150.76it/s]


1

In [14]:
import random
index = chunks.get_indexes()[random.randint(1, 50)]
print(len(chunks[index]))
chunks[index]

88


'Этот маленький ребус он послал нам с первой почтой, а сам выехал сюда ближайшим поездом.'

In [15]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex
from mirage import FaissIndexFlatL2, FaissIndexIVFPQR
vector_index = FaissIndexIVFPQR(dimensionality=embedder.get_dimensionality())
embedder.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 427/427 [00:43<00:00,  9.85it/s]


In [39]:
query = 'убитая женщина'
query_vector = embedder.embed(text=query)
# print(query_vector)
vector_answer = vector_index.query(
    query_vector, top_k=20
)
print(vector_answer)

[QueryResult(score=0.8335791230201721, chunk_storage_key='376f5652-13c7-4e6b-afb0-45bfbb531550', vector=array([ 0.01070768,  0.01809778, -0.03436125, ..., -0.01930608,
       -0.01730648,  0.04182574], dtype=float32)), QueryResult(score=0.8566122055053711, chunk_storage_key='e17fd7c4-af4e-4411-8db6-4273c7e5c9d0', vector=array([-0.0221016 ,  0.01742174, -0.07253658, ..., -0.06477947,
       -0.04252745,  0.03287837], dtype=float32)), QueryResult(score=0.8726161122322083, chunk_storage_key='9c27a6d0-ce9c-4aca-a3a0-71d5dfb97aac', vector=array([-0.00987871,  0.03530333, -0.04278317, ..., -0.03488935,
       -0.04346183,  0.03543985], dtype=float32)), QueryResult(score=0.8734219670295715, chunk_storage_key='9c084351-b6e4-46bd-9b40-3042d0ef9bb1', vector=array([ 0.01042651,  0.0605932 , -0.05291994, ...,  0.00316011,
        0.02510138,  0.02404919], dtype=float32)), QueryResult(score=0.8924689292907715, chunk_storage_key='418a5eff-8198-433d-970b-80cb6ed0c595', vector=array([-0.00151548,  0.0

In [40]:
from mirage.inference.quorums.RusVectoresQuorum import RusVectoresQuorum


quorum = RusVectoresQuorum(
    chunk_storage=chunks, 
    path_to_model='mirage\inference\quorums\model.bin',
    POS_thresholds={"ADJ": 0.5, "VERB": 0.65},
    visualize=True,
    max_combinations=25)
fulltext_search_answer = quorum.query(query)

Amount of vectors loaded in quorum: 189193
word "быть" is presented in word vectors: True
set of POS in w2v model: {'PROPN', 'X', 'NOUN', 'VERB', 'ADV', 'INTJ', 'ADJ', 'SYM', 'NUM'}
Rules of synonimization: {'ADJ': 0.5, 'VERB': 0.65}
 For word "убитая" obtained synonims:
[('убитый_VERB', 0.7544418573379517), ('убивать_NOUN', 0.6665365695953369), ('убитый_NOUN', 0.6586453318595886), ('убивать_ADJ', 0.6549819111824036), ('убить_ADJ', 0.6502050161361694), ('убить_VERB', 0.6136346459388733), ('труп_NOUN', 0.5793484449386597), ('убивать_VERB', 0.561595618724823), ('раненый_ADJ', 0.526347815990448), ('окровавленный_ADJ', 0.5030656456947327), ('окровавливать_VERB', 0.4972875118255615), ('окровавленный_VERB', 0.494367778301239), ('бездыхать_ADJ', 0.4913272559642792), ('бездыханный_ADJ', 0.4852805435657501), ('ранить_ADJ', 0.48326724767684937), ('раненый_NOUN', 0.47747743129730225), ('окровавить_ADJ', 0.4754990339279175), ('стрелявший_VERB', 0.46570515632629395), ('убийца_NOUN', 0.4600498676300

In [41]:
from mirage.inference.rerankers.LinearCombinationReranker import LinearCombinationReranker

rank_fusion = LinearCombinationReranker(fulltext_score_weight=0.25, vector_score_weight=1)
ranked_answer = rank_fusion(fulltext_search_answer, vector_answer)

In [42]:
chunks.get_texts_for_search_results(ranked_answer)

['— Вы трогали убитого? — спросил Холме.',
 '— Его несомненно закрыла женщина, закрыла инстинктивно…',
 'Вы кончили тем, что убили этого благородного человека и довели его жену до самоубийства.',
 'Показания обеих женщин были в высшей степени точны.',
 '— Она была найдена тяжело раненной возле своего мертвого мужа.',
 'Окно было закрыто на задвижку изнутри, обе женщины утверждали это с полной уверенностью.',
 'Мы не могли оставить ее, раненную, на полу.',
 'Мистер Хилтон Кьюбитт убит выстрелом прямо в сердце.',
 'Если же вы думаете, что я в состоянии ранить ту женщину, значит, вы не знаете ни ее, ни меня. Ни один мужчина никогда не любил ни одной женщины так, как я любил ее.',
 'Холме шарил в траве и листьях, как охотничий пес, разыскивающий раненую птицу.',
 'Затем, с помощью конюха и работающего на конюшне мальчишки, они отнесли свою раненую хозяйку в ее комнату.',
 '— Я стрелял в него, но и он стрелял в меня, — следовательно, это нельзя назвать убийством.',
 '— Отвратительное состоя

In [ ]:
from mirage.inference.prompters.APILLM import LLM
llm = LLM()
llm.do_request(query=query,
               chunk_storage=chunks,
               indexes=[element.chunk_storage_key for element in ranked_answer],
               prompt='')